In [1]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
from urllib.parse import urlencode 

In [2]:
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=True)
mars_data={}

In [3]:
def getSoup(url_to_get):
    url=url_to_get
    try:
        browser.visit(url)
        html = browser.html
        soup = bs(html, "html.parser")
        print("Scraping "+url)
        #print(len(html))
        return soup
    except:
        return ""

In [ ]:
'''
#Reference
try:
    r = requests.get(url, params={'s': thing})
except requests.exceptions.Timeout:
    # Maybe set up for a retry, or continue in a retry loop
except requests.exceptions.TooManyRedirects:
    # Tell the user their URL was bad and try a different one
except requests.exceptions.RequestException as e:
    # catastrophic error. bail.
    print e
    sys.exit(1)
    
    
with open('outfile.txt', 'w') as outfile:
    outfile.write(thisSoup)
print (type(thisSoup)) 

'''

In [4]:
#collect the latest News Title and Paragraph Text
thisSoup=getSoup("https://mars.nasa.gov/news/")


Scraping https://mars.nasa.gov/news/


In [5]:
if len(thisSoup) !=-1:
    news_title = thisSoup.find('li', class_='slide').find('a').find('h3').text
    
    news_paragraph=thisSoup.find('li', class_='slide').find('div', class_="article_teaser_body").text
    mars_data["news_title"]=news_title
    mars_data["news_paragraph"]=news_paragraph
else:
    mars_data["news_title"]="unavailable"
    mars_data["news_paragraph"]="unavailable"
    

In [6]:

print(mars_data)


{'news_title': 'NASA to Share Results of Effort to Recover Mars Rover', 'news_paragraph': 'NASA will discuss the status of its Mars Exploration Rover Opportunity in a media briefing at 11 a.m. PST (2 p.m. EST) Wednesday, Feb. 13, from the Jet Propulsion Laboratory.'}


In [7]:
#* Visit the url for JPL Featured Space Image [here](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars).
#find the image url for the current Featured Mars Image and assign the url string to a variable called `featured_image_url`.
thisUrl="https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
thisSoup=getSoup(thisUrl)
if len(thisSoup) !=1:
    featured_img=thisSoup.find('section',class_='centered_text clearfix main_feature primary_media_feature single').find('article',class_="carousel_item")
    featured_img_url=featured_img.get('style')
    featured_img_url=thisUrl[:24]+featured_img_url[23:-3]
    mars_data["featured_img_url"]=featured_img_url
else:
    mars_data["featured_img_url"]="unavailable"


Scraping https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars


In [8]:
print(mars_data)


{'news_title': 'NASA to Share Results of Effort to Recover Mars Rover', 'news_paragraph': 'NASA will discuss the status of its Mars Exploration Rover Opportunity in a media briefing at 11 a.m. PST (2 p.m. EST) Wednesday, Feb. 13, from the Jet Propulsion Laboratory.', 'featured_img_url': 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA17440-1920x1200.jpg'}


In [9]:
#* Visit the Mars Weather twitter account [here](https://twitter.com/marswxreport?lang=en) and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called `mars_weather`.
thisSoup=getSoup("https://twitter.com/marswxreport?lang=en")
if thisSoup !=-1:
    for div in thisSoup.find_all('div',attrs={'data-name':'Mars Weather'}):
        tweet= div.find('p', class_='tweet-text')
        if tweet and tweet.text.find('Sol ')!= -1:
            mars_weather= tweet.text.strip().replace('\n',' ')            
            mars_weather=mars_weather[:mars_weather.find("pic")]
            break
    mars_data["mars_weather"]=mars_weather
else:
    mars_data["mars_weather"]="unavailable"



Scraping https://twitter.com/marswxreport?lang=en


In [10]:
print(mars_data)


{'news_title': 'NASA to Share Results of Effort to Recover Mars Rover', 'news_paragraph': 'NASA will discuss the status of its Mars Exploration Rover Opportunity in a media briefing at 11 a.m. PST (2 p.m. EST) Wednesday, Feb. 13, from the Jet Propulsion Laboratory.', 'featured_img_url': 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA17440-1920x1200.jpg', 'mars_weather': 'Sol 2315 (2019-02-09), high -17C/1F, low -71C/-95F, pressure at 8.13 hPa, daylight 06:47-18:52'}


In [11]:
#* Visit the Mars Facts webpage [here](http://space-facts.com/mars/) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
#* Use Pandas to convert the data to a HTML table string.

table_df=pd.read_html("http://space-facts.com/mars/")
df=table_df[0]
df.columns=["Fact","Value"]
df.columns.index=["Fact"]
htmlStr=df.to_html
if htmlStr:
    mars_data["facts"]=htmlStr
else:
    mars_data["facts"]="unavailable"


In [12]:
print(mars_data)

{'news_title': 'NASA to Share Results of Effort to Recover Mars Rover', 'news_paragraph': 'NASA will discuss the status of its Mars Exploration Rover Opportunity in a media briefing at 11 a.m. PST (2 p.m. EST) Wednesday, Feb. 13, from the Jet Propulsion Laboratory.', 'featured_img_url': 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA17440-1920x1200.jpg', 'mars_weather': 'Sol 2315 (2019-02-09), high -17C/1F, low -71C/-95F, pressure at 8.13 hPa, daylight 06:47-18:52', 'facts': <bound method DataFrame.to_html of                    Fact                          Value
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.52 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                  -153 to 20 °C
7         First Record:           

In [13]:
### Mars Hemispheres
#* Visit the USGS Astrogeology site [here](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution images for each of Mar's hemispheres.
thisSoup=getSoup("https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars")
if thisSoup!=-1:
    enh_img=thisSoup.find('div', class_='downloads').find('a')
else:
    print("error getting soup")


Scraping https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars


AttributeError: 'NoneType' object has no attribute 'find'

In [ ]:
#click each of the links to the hemispheres in order to find the image url to the full resolution image.

In [ ]:
#* Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys `img_url` and `title`.


In [ ]:
#* Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.